In [97]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
import ptsa
import sys
import os
import matplotlib.pyplot as plt
from pylab import *
from copy import copy
from scipy import stats
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
# sys.path.append('/home1/john/SWRrefactored/code/SWR_modules/')
sys.path.insert(0, '/home1/john/SWRrefactored/code/SWR_modules/')
from SWRmodule import *
from general import * #superVstack,findInd,findAinB
import csv
import os
import dill, pickle
import mne
from copy import copy    
from scipy.signal import firwin,filtfilt,kaiserord
from ptsa.data.filters import ButterworthFilter, ResampleFilter, MorletWaveletFilter
import xarray as xarray
from brain_labels import HPC_labels, ENT_labels, PHC_labels, temporal_lobe_labels,\
                        MFG_labels, IFG_labels, nonHPC_MTL_labels, ENTPHC_labels, AMY_labels, ACC_OF_MFC_labels
import pingouin as pg

################################################################
df = get_data_index("r1") # all RAM subjects

### CHOOSE HERE
exp = 'catFR1' # 'catFR1' #'FR1'
selected_period = 'countdown' 
# working trial periods: # surrounding_recall # encoding # countdown # distractor
available_regions = {
#     "ACC_OF_labels": ACC_OF_MFC_labels,   
    "HPC_labels": HPC_labels,
#     "ENTPHC_labels": ENTPHC_labels,
#     "AMY_labels": AMY_labels
}

remove_soz_ictal = 0
recall_minimum = 2000 #5000 # 2000
filter_type = 'hamming'
extra = '' #'- ZERO_IRI'

save_path = f'/scratch/john/SWRrefactored/patient_info/{exp}/'
brain_region_idxs = np.arange(len(available_regions))
if selected_period == 'encoding':
    recall_type_switch = 10 # no IRI requirements for recalls that is necessary for surrounding_recall
elif selected_period == 'surrounding_recall':
    recall_type_switch = 0 # 0 for original (remove IRI < recall_minimum), 1 for only those with subsequent, 2 for second recalls only, 3 for isolated recalls
else:
    recall_type_switch = 10
    print('make sure you read through and understand what getSWRpathInfo is doing')
################################################################

make sure you read through and understand what getSWRpathInfo is doing


In [123]:
selected_region = HPC_labels

In [98]:
# def ClusterRunSWRs(row, selected_region,save_path, selected_period, 
#                    exp, testing_mode=False):

'''
:param [int, str] temp_df_select: 
    if int, then temp_df_select is used to index a row of temp_df
    if str, then must be name of a patient. function iterates through 
    rows of temp_df until row.subject equals temp_df_select

:param str selected_period: encoding, surrounding_recall, or whole_retrieval
:param str selected_region: which brain region to generate data for 
:param str save_path: folder to save data in
:param str exp: catFR or FR
:param bool testing_mode: if in testing mode, saves data to "test.p" 
so as to not overwrite existing data
'''

### PARAMS ###

##
## NOTE: for looking at individual lags make sure min_recalls is set to 0 or will eliminate trials from low recall lists
##

save_values = 1

# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
  # switch to determine which recalls to look at! (see details below)
# (as of 2021 always leave this as 0, since I select for 4/6/etc below)
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur 
# 2: test condition where we look at second recalls within IRI ONLY (there is an initial recall in 2 s before current recall)
# 3: isolated recalls with no other recalls +/- RECALL_MINIMUM s
# 4: only first recall of every retrieval period
# 5: take only those recalls that come second in retrieval period within 2 s of first retrieval
# 6: take only NOT first recall of every retrieval period
# 7: take only NOT first recall AND ISOLATED trials (this should REALLY maximize SWR bump)
# 10: same as 0 but with no IRI (mostly just to see number of recalls)
remove_soz_ictal = 0 # 0 for nothing, 1 for SOZ, 2 for SOZ+ictal    
min_ripple_rate = 0.025 # Hz. # 0.1
max_ripple_rate = 1.5 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2

### semantic/temporal clustering parameters ###
min_recalls = 0
PCA_ndim = 1 # number of PC dims to use for semantic clustering (Ethan usually found only 1 worked for theta/FC)

#     # for parametric run through recall_minimums
#         recall_mins = np.arange(800,5100,100) #[800,900,1100,1200,1300,1400]
# #     recall_mins = [1600,1700,1800,1900,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000]
#     for recall_minimum in recall_mins:

# recall params
#     recall_minimum = 5000 # for isolated recalls what is minimum time for recall to be considered isolated?
IRI = 2000 #5000 # inter-recall interval...remove recalls within this range (keep only first one and remove those after it)
retrieval_whole_time = 10000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 1000 # buffer to add to either end of IRI when processing eeg #**
filter_type = 'hamming' # see local version below for details. 'butter' is removed from cluster version so don't change this    

### END PARAMS ###    

# get region label
if selected_region == HPC_labels:
    region_name = 'HPC'
elif selected_region == ENT_labels:
    region_name = 'ENT'
elif selected_region == PHC_labels:
    region_name = 'PHC'
elif selected_region == temporal_lobe_labels:
    region_name = 'TEMPORALLOBE'
elif selected_region == MFG_labels:
    region_name = 'MFG'
elif selected_region == IFG_labels:
    region_name = 'IFG'
elif selected_region == nonHPC_MTL_labels:
    region_name = 'nonHPC_MTL'
elif selected_region == ENTPHC_labels:
    region_name = 'ENTPHC'
elif selected_region == AMY_labels:
    region_name = 'AMY'
elif selected_region == ACC_OF_MFC_labels:
    region_name = 'ACC_OF'

if selected_period == 'surrounding_recall':
    if IRI == 0:
        psth_start = -2000
        psth_end = 2000
    else:
        psth_start = -IRI # only makes sense to look at period <= IRI
        psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time
elif selected_period == 'countdown':
    psth_start = 0
elif selected_period == 'distractor':
    psth_start = 0

ripple_array = []; fr_array = []; 
trial_nums = []; 
elec_names = []; sub_sess_names = []; sub_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
align_adjust = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = []
encoded_word_key_array = []; category_array = []

session_ripple_rate_by_elec = []; # for cluster version only since I compare across electrodes for each session

list_recall_num_array = []; rectime_array = []; # new ones added 2020-11-24
serialpos_array = [] # used to be encoding info but commandeered for surrounding_recalls ~~~
recall_position_array = []
session_events = pd.DataFrame()

trial_by_trial_correlation = []; elec_by_elec_correlation = []
elec_ripple_rate_array = []
channel_coords = []; electrode_labels = []

semantic_clustering_key = []
temporal_clustering_key = []
semantic_clustering_from_key = []
serialpos_lags = []; serialpos_from_lags = []; CRP_lags = []
list_trial_nums = []; list_num_key = []
list_level_semantic = []; list_level_temporal = []

program_ran = 0


with open(f'/scratch/john/SWRrefactored/patient_info/temp_dfSWR_encoding_{exp}.p', 'rb') as f: ### change here to avoid overwrite
    temp_df = dill.load(f)


'\n:param [int, str] temp_df_select: \n    if int, then temp_df_select is used to index a row of temp_df\n    if str, then must be name of a patient. function iterates through \n    rows of temp_df until row.subject equals temp_df_select\n\n:param str selected_period: encoding, surrounding_recall, or whole_retrieval\n:param str selected_region: which brain region to generate data for \n:param str save_path: folder to save data in\n:param str exp: catFR or FR\n:param bool testing_mode: if in testing mode, saves data to "test.p" \nso as to not overwrite existing data\n'

In [122]:
max_sess = len(temp_df)

for num in range(0,max_sess):
    row = temp_df[num]

    sub = row.subject
    session = row.session
    exp = row.experiment
    mont = int(row.montage)
    loc = int(row.localization)
    sub_sess = f"{sub}-{session}"  # for convenience

    reader = CMLReadDFRow(row)
    
    # get localizations (region info)
    pairs = reader.load('pairs')
    try:
        localizations = reader.load('localization')
    except:
        localizations = []
    tal_struct, bipolar_pairs, mpchans = get_bp_tal_struct(sub, montage=mont, localization=loc)
    elec_regions,atlas_type,pair_number,has_stein_das = get_elec_regions(localizations,pairs) 

    evs = reader.load("task_events")
    
    if selected_period == 'countdown':

        # -------------------------------------------------
        # Check if COUNTDOWN_START or COUNTDOWN exist
        # -------------------------------------------------
        if "COUNTDOWN_START" not in evs["type"].values:
            if "COUNTDOWN" not in evs["type"].values:
                print(f"{sub_sess} does NOT contain COUNTDOWN or COUNTDOWN_START.")
            else:
                # Proceed with new countdown pivot/duration checks
                countdown2 = evs.query("type in ['COUNTDOWN','ENCODING_START']")
                countdown2_pivot = countdown2.pivot_table(
                    index="list",
                    columns="type",
                    values="mstime",
                    aggfunc="first"
                )
                countdown2_pivot["countdown2_duration"] = (
                    countdown2_pivot["ENCODING_START"] - countdown2_pivot["COUNTDOWN"]
                )

                # Valid new countdown durations
                good_countdown2_mask = (
                    countdown2_pivot["countdown2_duration"].notna() &
                    (countdown2_pivot["countdown2_duration"] >= 0)
                )
                good_lists = countdown2_pivot.index[good_countdown2_mask]

                # Find minimum good new countdown duration
                min_duration = countdown2_pivot.loc[
                    good_lists,
                    "countdown2_duration"
                ].min()

                # Check for "bad" new countdown threshold
                if min_countdown2_duration < 9995:
                    print(f"{sub_sess}")
                    print("BAD NEW COUNTDOWN")
                    print(min_duration)
                    
                # Filter to final events for good lists
                final_evs = evs[
                    evs["list"].isin(good_lists)
                    & evs["type"].isin(["COUNTDOWN"])
                ]    
        else:
            # Proceed with countdown pivot/duration checks
            countdown = evs.query("type in ['COUNTDOWN_START','COUNTDOWN_END']")
            countdown_pivot = countdown.pivot_table(
                index="list", 
                columns="type", 
                values="mstime", 
                aggfunc="first"
            )
            countdown_pivot["countdown_duration"] = (
                countdown_pivot["COUNTDOWN_END"] - countdown_pivot["COUNTDOWN_START"]
            )

            # Valid countdown durations
            good_countdown_mask = (
                countdown_pivot["countdown_duration"].notna() &
                (countdown_pivot["countdown_duration"] >= 0)
            )
            good_lists = countdown_pivot.index[good_countdown_mask]
            
            # a few lists have eegoffset<=0 which breaks things
            bad_eegoffset_lists = evs.loc[evs["eegoffset"] <= 0, "list"].unique()
            good_lists = [lst for lst in good_lists if lst not in bad_eegoffset_lists] # filter out


            # Find minimum good countdown duration
            min_duration = countdown_pivot.loc[
                good_lists, 
                "countdown_duration"
            ].min()

            # Check for "bad" countdown threshold
            if min_duration < 9995:
                print(f"{sub_sess}")
                print("BAD COUNTDOWN")
                print(min_duration)
            # Filter to final events for good lists
            final_evs = evs[
                evs["list"].isin(good_lists)
                & evs["type"].isin(["COUNTDOWN_START"])
            ]    

    elif selected_period == 'distractor':
        # -------------------------------------------------
        # Check if DISTRACT_START exists
        # -------------------------------------------------
        if "DISTRACT_START" not in evs["type"].values:
            print(f"{sub_sess} does NOT contain a distractor period.")
        else:
            # Proceed with distractor pivot/duration checks
            distractor = evs.query("type in ['DISTRACT_START','DISTRACT_END']")
            distractor_pivot = distractor.pivot_table(
                index="list", 
                columns="type", 
                values="mstime", 
                aggfunc="first"
            )
            distractor_pivot["distract_duration"] = (
                distractor_pivot["DISTRACT_END"] - distractor_pivot["DISTRACT_START"]
            )

            # Valid distractor durations
            good_distractor_mask = (
                distractor_pivot["distract_duration"].notna() &
                (distractor_pivot["distract_duration"] >= 0)
            )
            good_lists = distractor_pivot.index[good_distractor_mask]

            # Find minimum good distractor duration
            min_duration = distractor_pivot.loc[
                good_lists, 
                "distract_duration"
            ].min()

            # Check for "bad" distractor threshold
            if min_duration < 19995:
                print(f"{sub_sess}")
                print("BAD DISTRACTOR")
                print(min_duration)
            # Filter to final events for good lists
            final_evs = evs[
                evs["list"].isin(good_lists)
                & evs["type"].isin(["DISTRACT_START"])
            ]                

    # outputs to save: final_evs, good_lists, and min_duration
    psth_end = min_duration

R1302M-0
BAD COUNTDOWN
0
R1320D-1
BAD COUNTDOWN
0
R1332M-1
BAD COUNTDOWN
0
R1390M-0
BAD COUNTDOWN
250
R1390M-1
BAD COUNTDOWN
2067
R1390M-2
BAD COUNTDOWN
2717
R1390M-3
BAD COUNTDOWN
2784
R1401J-2
BAD COUNTDOWN
35
R1414E-0
BAD COUNTDOWN
17
R1441T-0
BAD COUNTDOWN
132
R1457T-0
BAD COUNTDOWN
616
R1495J-4
BAD COUNTDOWN
399
R1526J-0
BAD COUNTDOWN
3700
R1536J-3
BAD COUNTDOWN
650


FileNotFoundError: Unable to find the requested file in any of the expected locations:
 /protocols/r1/subjects/R1696L/experiments/catFR1/sessions/0/behavioral/current_processed/task_events.json
/data/events/pyFR/R1696L_events.mat

In [ ]:
# load eeg
evs_encoding_words = evs_encoding_words[evs_encoding_words.eegoffset>-1]

# fixing bad trials
if sub == 'R1045E' and exp == 'FR1': # this one session has issues in eeg trials past these points so remove events
    if selected_period == 'whole_retrieval':
        eeg_events = eeg_events.iloc[:20,:] # only the first 20 retrieval periods have good eeg
    elif selected_period == 'encoding':
        eeg_events = eeg_events.iloc[:263,:] # same idea
        encoded_word_key = encoded_word_key[:263]
    else:
        sys.exit() # too difficult to deal with for clustering analysis   
        # eeg_events = eeg_events.iloc[:65,:] # only the first 66 recalls have good eeg

# note I added the align_adjust now for whole_retrieval where I adjust all retrieval starts to beep_end
align_adjust = 0
eeg = reader.load_eeg(events=eeg_events, rel_start=psth_start-eeg_buffer+align_adjust, 
                      rel_stop=psth_end+eeg_buffer+align_adjust, clean=True, scheme=pairs) 